# 📊 접근 로그 분석 노트북

**분석 대상**: `suwon_outer` (수원시 외부 포털 접근 로그)

---

## 1. 환경 설정

In [2]:
# ===== Autoreload 설정 =====
%load_ext autoreload
%autoreload 2

# ===== 기본 라이브러리 =====
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# ===== DB 유틸리티 =====
from db_utils import get_dataframe, list_databases, get_tables, preview, save_dataframe

# ===== 시각화 설정 (Robust Font Fix) =====
import matplotlib.font_manager as fm
import matplotlib as mpl

font_path = '/usr/share/fonts/noto/NotoSansCJKkr-Regular.otf'
if os.path.exists(font_path):
    # 폰트 등록 (Matplotlib 3.2+ 호환 방식)
    fm.fontManager.addfont(font_path)
    prop = fm.FontProperties(fname=font_path)
    plt.rcParams['font.family'] = prop.get_name()
    print(f"Font successfully set to: {prop.get_name()}")
else:
    print("Warning: Font file not found. Falling back to default.")

plt.rcParams['axes.unicode_minus'] = False
sns.set_theme(style='whitegrid', font=plt.rcParams['font.family'])

# 1. 폰트 파일 직접 등록
font_path = '/usr/share/fonts/noto/NotoSansCJKkr-Regular.otf'
if os.path.exists(font_path):
    fm.fontManager.addfont(font_path)
    font_name = fm.FontProperties(fname=font_path).get_name()
    plt.rc('font', family=font_name)
else:
    # 대체 폰트 설정
    plt.rc('font', family='Noto Sans CJK KR')

plt.rcParams['axes.unicode_minus'] = False
sns.set_theme(style='whitegrid', font=plt.rcParams['font.family'])


# ===== pandas 출력 설정 =====
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

Font successfully set to: Noto Sans CJK KR


In [3]:
# ========================================
# 🔧 이 셀만 수정하면 다른 DB로 전환 가능
# ========================================
DB_ALIAS = "suwon_portal"  # db_config.json에 정의된 DB 별칭

# 사용 가능한 DB 목록 확인
list_databases()

사용 가능한 데이터베이스 목록

📦 koroad_portal
   유형: postgresql
   DB명: da
   설명: 한국도로공사 내부 포털 접근 로그 DB

📦 koroad_kiosk
   유형: mariadb
   DB명: koroad
   설명: 한국도로공사 키오스크 접근 로그 DB

📦 suwon_platform
   유형: postgresql
   DB명: suwon
   설명: 수원시 내부 포털 접근 로그 DB

📦 suwon_portal
   유형: postgresql
   DB명: postgres
   설명: 수원시 외부 포털 접근 로그 DB



## 2. 데이터 로드

In [4]:
# 테이블 목록 확인
tables = get_tables(DB_ALIAS,schema='idp')
tables

/DATA/jupyter_WorkingDirectory/notebook/Access_Analysis/db_utils.py:243: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,table_name,table_type
0,cm_analysis_case,BASE TABLE
1,cm_attach_file,BASE TABLE
2,cm_attaching_file_group,BASE TABLE
3,cm_board_category,BASE TABLE
4,cm_board_comment,BASE TABLE
5,cm_board_file,BASE TABLE
6,cm_board_master,BASE TABLE
7,cm_board_post,BASE TABLE
8,cm_data_source,BASE TABLE
9,cm_group,BASE TABLE


In [5]:
# ===== 데이터 로드 (쿼리 수정) =====
query = """
SELECT *
FROM mn_log_audit

"""

df = get_dataframe(DB_ALIAS, query,schema='idp')
print(f"로드된 데이터: {df.shape[0]:,}행 x {df.shape[1]}열")
df.head()
df['issued'] = pd.to_datetime(df['ins_dt'])

/DATA/jupyter_WorkingDirectory/notebook/Access_Analysis/db_utils.py:243: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


DatabaseError: Execution failed on sql '
SELECT *
FROM mn_log_audit

': relation "mn_log_audit" does not exist
LINE 3: FROM mn_log_audit
             ^


## 3. 기초 탐색 (EDA)

In [ ]:
# 데이터 타입 및 결측치 확인
df.info()

In [ ]:
# 기술통계량
df.describe(include='all')

## 4. 분석

In [ ]:
# 여기에 분석 코드 작성


## 4. 접속 지표 (Proxy AU by IP)

In [ ]:
# 일일/주간/월간 고유 IP 기반 활동성 계산
df['date'] = df['issued'].dt.date
df['week'] = df['issued'].dt.to_period('W').apply(lambda r: r.start_time)

dip = df.groupby('date')['ip'].nunique()
wip = df.groupby('week')['ip'].nunique()

print(f"평균 일일 고유 IP: {dip.mean():.1f}")
print(f"평균 주간 고유 IP: {wip.mean():.1f}")

plt.figure(figsize=(15, 6))
plt.plot(dip.index, dip.values, label='Unique IP (Daily)', alpha=0.6)
plt.title('수원시 외부 포털 고유 IP 추세', fontsize=15)
plt.ylabel('고유 IP 수')
plt.show()

## 5. 인기 콘텐츠 및 경로 분석

In [ ]:
# 가장 많이 접근한 URL 상위 10개
top_urls = df['url'].value_counts().head(10)

plt.figure(figsize=(12, 6))
sns.barplot(x=top_urls.values, y=top_urls.index, hue=top_urls.index, palette='viridis', legend=False)
plt.title('외부 포털 인기 페이지 TOP 10', fontsize=15)
plt.xlabel('접근 횟수')
plt.show()